# Примеры использования модуля datasets

Этот ноутбук демонстрирует работу с модулем `mas_automl.datasets` для загрузки и управления датасетами из OpenML.

In [12]:
# Импорт необходимых библиотек
from mas_automl.datasets import DatasetManager, DatasetType, OpenMLDatasetMetadata
import pandas as pd


## 1. Инициализация менеджера датасетов

Создаём экземпляр `DatasetManager`, который будет управлять датасетами из OpenML и локальными файлами.


In [13]:
# Создаём менеджер датасетов
# cache_openml=True означает, что скачанные датасеты будут сохранены локально
manager = DatasetManager(cache_openml=True)

print("Менеджер датасетов инициализирован!")
print(f"Директория для сырых данных: {manager.raw_data_dir}")
print(f"Директория метаданных: {manager.metadata_dir}")


Менеджер датасетов инициализирован!
Директория для сырых данных: C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets
Директория метаданных: C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\.metadata


## 2. Получение метаданных датасета по OpenML ID

Получим метаданные датасета `credit-g` (ID: 31) из OpenML.

In [14]:
# Получаем метаданные датасета по ID
# download=False означает, что мы пока не скачиваем данные, только метаданные
manager.get_openml_dataset(31, download=False)
# Метаданные доступны через manager.metadata или manager.get_metadata()
metadata = manager.metadata

print(f"Название: {metadata.name}")
print(f"OpenML ID: {metadata.dataset_id}")
print(f"Версия: {metadata.version}")
print(f"Тип задачи: {metadata.dataset_type}")
print(f"Целевая переменная: {metadata.default_target_attribute}")
print(f"Количество строк: {metadata.num_rows}")
print(f"Количество признаков: {metadata.num_features}")
print(f"Количество классов: {metadata.num_classes}")
print(f"Описание: {metadata.description[:200] if metadata.description else 'Нет описания'}...")
print(f"Теги: {metadata.tags}")


Название: credit-g
OpenML ID: 31
Версия: 1
Тип задачи: DatasetType.CLASSIFICATION
Целевая переменная: class
Количество строк: 1000
Количество признаков: 21
Количество классов: 2
Описание: **Author**: Dr. Hans Hofmann  
**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) - 1994    
**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy....
Теги: []


## 3. Просмотр всех метаданных

Посмотрим все доступные метаданные из OpenML:


In [15]:
# Выводим все метаданные в удобном формате
from rich.console import Console
console = Console()

# Используем компактный вывод без лишних пустых строк
output_lines = [
    "[bold cyan]Полные метаданные датасета:[/bold cyan]",
    f"  [green]ID:[/green] {metadata.dataset_id}",
    f"  [green]Название:[/green] {metadata.name}",
    f"  [green]Создатель:[/green] {metadata.creator}",
    f"  [green]Дата загрузки:[/green] {metadata.upload_date}",
    f"  [green]Лицензия:[/green] {metadata.licence}",
    f"  [green]URL:[/green] {metadata.url}",
    f"  [green]Цитирование:[/green] {metadata.citation[:100] if metadata.citation else 'Нет'}...",
    f"  [green]Пропущенные значения:[/green] {metadata.num_missing_values}",
]

# Метрики качества
if metadata.quality:
    output_lines.append("\n[bold yellow]Метрики качества:[/bold yellow]")
    for key, value in list(metadata.quality.items())[:10]:  # Показываем первые 10
        output_lines.append(f"  {key}: {value}")

# Выводим всё одной командой
console.print("\n".join(output_lines))


Полные метаданные датасета:
  ID: 31
  Название: credit-g
  Создатель: Dr. Hans Hofmann
  Дата загрузки: 2014-04-06 23:21:47
  Лицензия: Public
  URL: https://api.openml.org/data/v1/download/31/credit-g.arff
  Цитирование: https://archive.ics.uci.edu/ml/citation_policy.html...
  Пропущенные значения: 0

Метрики качества:
  AutoCorrelation: 0.5695695695695696
  CfsSubsetEval_DecisionStumpAUC: 0.7208285714285714
  CfsSubsetEval_DecisionStumpErrRate: 0.273
  CfsSubsetEval_DecisionStumpKappa: 0.3056968463886062
  CfsSubsetEval_NaiveBayesAUC: 0.7208285714285714
  CfsSubsetEval_NaiveBayesErrRate: 0.273
  CfsSubsetEval_NaiveBayesKappa: 0.3056968463886062
  CfsSubsetEval_kNN1NAUC: 0.7208285714285714
  CfsSubsetEval_kNN1NErrRate: 0.273
  CfsSubsetEval_kNN1NKappa: 0.3056968463886062

## 4. Получение датасета по имени

Можно также получить датасет по его имени:


In [5]:
# Получаем датасет по имени
manager.get_openml_dataset("diabetes", download=False)
# Метаданные доступны через manager.metadata
diabetes_metadata = manager.metadata

print(f"Название: {diabetes_metadata.name}")
print(f"OpenML ID: {diabetes_metadata.dataset_id}")
print(f"Тип задачи: {diabetes_metadata.dataset_type}")
print(f"Целевая переменная: {diabetes_metadata.default_target_attribute}")


Название: diabetes
OpenML ID: 37
Тип задачи: DatasetType.CLASSIFICATION
Целевая переменная: class


## 5. Загрузка данных датасета

Теперь загрузим сами данные датасета в pandas DataFrame:


In [6]:
# Загружаем данные датасета
# download=True означает, что данные будут скачаны из OpenML
# cache=True означает, что данные будут сохранены локально для повторного использования
df = manager.get_openml_dataset(31, download=True, cache=True)

print(f"Размер датасета: {df.shape}")
print(f"\nПервые 5 строк:")
print(df.head())
print(f"\nИнформация о датасете:")
print(df.info())

# Метаданные также доступны после загрузки данных
print(f"\nМетаданные доступны через manager.metadata:")
print(f"  Название: {manager.metadata.name}")
print(f"  Тип задачи: {manager.metadata.dataset_type}")


Размер датасета: (1000, 21)

Первые 5 строк:
  checking_status  duration                  credit_history  \
0              <0         6  critical/other existing credit   
1        0<=X<200        48                   existing paid   
2     no checking        12  critical/other existing credit   
3              <0        42                   existing paid   
4              <0        24              delayed previously   

               purpose  credit_amount    savings_status employment  \
0             radio/tv         1169.0  no known savings        >=7   
1             radio/tv         5951.0              <100     1<=X<4   
2            education         2096.0              <100     4<=X<7   
3  furniture/equipment         7882.0              <100     4<=X<7   
4              new car         4870.0              <100     1<=X<4   

   installment_commitment     personal_status other_parties  ...  \
0                       4         male single          none  ...   
1                  

In [16]:
manager.metadata.local_path

WindowsPath('C:/Users/User1/Desktop/ITMO_bootcamp/data/datasets/openml_31_credit-g.csv')

In [8]:
import json
from datetime import datetime
from pathlib import Path


# Функция для преобразования объектов в JSON-совместимый формат
def json_serializer(obj):
    """Преобразует datetime и Path объекты в строки для JSON."""
    if isinstance(obj, datetime):
        return obj.isoformat()
    if isinstance(obj, Path):
        return str(obj)
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")


# Получаем данные метаданных и преобразуем их для JSON
metadata_dict = manager.metadata.model_dump()

# Преобразуем datetime и Path объекты в строки
if metadata_dict.get("upload_date") and isinstance(metadata_dict["upload_date"], datetime):
    metadata_dict["upload_date"] = metadata_dict["upload_date"].isoformat()
if metadata_dict.get("local_path") and isinstance(metadata_dict["local_path"], Path):
    metadata_dict["local_path"] = str(metadata_dict["local_path"])

with open('example.json', 'w') as f:
    json.dump(metadata_dict, f, ensure_ascii=False, indent=4, default=json_serializer)

## 6. Получение информации для передачи в другие модули

Метод `get_dataset_info` возвращает структурированную информацию, которую можно передать в другие модули системы:


In [9]:
# Получаем информацию о датасете для передачи в другие модули
# get_dataset_info возвращает полные метаданные (OpenMLDatasetMetadata)
dataset_info = manager.get_dataset_info(31)

print(f"Название: {dataset_info.name}")
print(f"OpenML ID: {dataset_info.dataset_id}")
print(f"Тип задачи: {dataset_info.dataset_type}")
print(f"Целевая переменная: {dataset_info.default_target_attribute}")
print(f"Локальный путь: {dataset_info.local_path}")
print(f"\nВсе метаданные доступны:")
print(f"  - Количество строк: {dataset_info.num_rows}")
print(f"  - Количество признаков: {dataset_info.num_features}")
print(f"  - Количество классов: {dataset_info.num_classes}")
print(f"  - Метрики качества: {len(dataset_info.quality)} метрик")


Название: credit-g
OpenML ID: 31
Тип задачи: DatasetType.CLASSIFICATION
Целевая переменная: class
Локальный путь: C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\openml_31_credit-g.csv

Все метаданные доступны:
  - Количество строк: 1000
  - Количество признаков: 21
  - Количество классов: 2
  - Метрики качества: 107 метрик


## 8. Пример использования в пайплайне

Покажем, как использовать информацию о датасете для передачи в другие модули системы:


In [10]:
# Получаем информацию о датасете для использования в пайплайне
# Сначала загружаем метаданные
manager.get_openml_dataset("credit-g", download=False)
# Затем получаем их через get_dataset_info или напрямую через manager.metadata
dataset_info = manager.get_dataset_info()  # Без параметров - возвращает последние метаданные

# Эта информация может быть передана в другие модули, например, в PlannerAgent
print("Информация для передачи в пайплайн:")
print(f"  Название: {dataset_info.name}")
print(f"  OpenML ID: {dataset_info.dataset_id}")
print(f"  Тип задачи: {dataset_info.dataset_type.value if dataset_info.dataset_type else 'unknown'}")
print(f"  Целевая переменная: {dataset_info.default_target_attribute}")
print(f"  Путь к данным: {dataset_info.local_path}")

# Можно также использовать напрямую manager.metadata
print(f"\nДополнительная информация (через manager.metadata):")
print(f"  Количество строк: {manager.metadata.num_rows}")
print(f"  Количество признаков: {manager.metadata.num_features}")
print(f"  Теги: {manager.metadata.tags}")


Информация для передачи в пайплайн:
  Название: credit-g
  OpenML ID: 31
  Тип задачи: classification
  Целевая переменная: class
  Путь к данным: C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\openml_31_credit-g.csv

Дополнительная информация (через manager.metadata):
  Количество строк: 1000
  Количество признаков: 21
  Теги: []


## 9. Работа с локальными датасетами (опционально)

Если у вас есть локальные файлы датасетов, их можно зарегистрировать в менеджере:


In [11]:
# Пример регистрации локального датасета (если у вас есть CSV файл)
# Раскомментируйте и укажите путь к вашему файлу:

# local_metadata = manager.register_dataset(
#     name="my_local_dataset",
#     file_path="path/to/your/dataset.csv",
#     description="Описание локального датасета",
#     target_column="target",
#     dataset_type=DatasetType.CLASSIFICATION,
#     auto_detect=True
# )
# 
# print(f"Зарегистрирован локальный датасет: {local_metadata.name}")
# print(f"Путь: {local_metadata.path}")
# print(f"Размер: {local_metadata.num_rows} строк, {local_metadata.num_features} признаков")


## Заключение

Модуль `mas_automl.datasets` предоставляет удобный интерфейс для работы с датасетами из OpenML:

- ✅ Получение метаданных по ID или имени через `get_openml_dataset(download=False)`
- ✅ Доступ к метаданным через `manager.metadata` или `manager.get_metadata()` без повторного указания ID
- ✅ Загрузка данных в pandas DataFrame через `get_openml_dataset(download=True)`
- ✅ Автоматическое кэширование датасетов и метаданных
- ✅ Полные метаданные через `get_dataset_info()` для передачи в другие модули
- ✅ Поддержка локальных датасетов

Все метаданные сохраняются локально и доступны для повторного использования без повторных запросов к OpenML API.
